In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from utils.nn.optimizer import Adam
from utils.nn.neuron import Activation
from utils.nn.layer import Dense, Dropout
from utils.nn.sequential import Sequential
from utils.nn.metrics import variables_to_float
from utils.nn.losses import mean_squared_error_loss
from utils.nn.initializer import Initializer, InitializationType


# Universal Function Approximation

In [ ]:
class FuncApproximator:
    """
    A class to approximate a scalar function using a FeedForward Neural Network.

    Attributes:
        model (FeedForwardNN): The neural network model to be trained.
        scalar_func (callable): The function to be approximated.
        x (np.ndarray): The input data points.
        y (np.ndarray): The output data points corresponding to x.
    """

    def __init__(self, model: Sequential, scalar_func: callable, x: np.ndarray, y: np.ndarray):
        """
        Initializes the FuncApproximator.

        Args:
            model (FeedForwardNN): An instance of the FeedForwardNN class.
            scalar_func (callable): The scalar function to approximate.
            x (np.ndarray): An array of input values.
            y (np.ndarray): An array of output values from the scalar_func.
        """
        self.model = model
        self.scalar_func = scalar_func
        self.x = x
        self.y = y
        self.y_pred = None  # To store predictions after fitting

    def fit(self, batch_size: int, test_split_size: float, epochs: int, display_on_each_n_step: int, learning_rate: float = 0.001):
        """
        Trains the neural network model to approximate the function.

        Args:
            batch_size (int): The number of samples per batch.
            test_split_size (float): The proportion of the dataset to include in the validation split.
            epochs (int): The number of epochs to train for.
            display_on_each_n_step (int): How often to display training progress.
            learning_rate (float, optional): The learning rate for the optimizer. Defaults to 0.001.
        """
        # Reshape data for train_test_split
        X_data_reshaped = self.x.reshape(-1, 1)
        self.y = self.y.reshape(-1, 1)#make it matrix

        # Split data into training and validation sets
        x_train_np, x_validate_np, y_train_np, y_validate_np = train_test_split(
            X_data_reshaped, self.y, test_size=test_split_size, random_state=42
        )

        # Convert to list of lists for X and list for y, as expected by FeedForwardNN
        x_train = x_train_np.tolist()
        y_train = y_train_np.tolist()
        x_validate = x_validate_np.tolist()
        y_validate = y_validate_np.tolist()

        # Choose Optimizer and Loss
        params = self.model.parameters()
        optimizer = Adam(params=params, learning_rate=learning_rate)
        loss_fn = mean_squared_error_loss

        # Fit the model
        self.model.fit(
            x_train=x_train,
            y_train=y_train,
            optimizer=optimizer,
            loss_func=loss_fn,
            epochs=epochs,
            batch_size=batch_size,
            metric="mse",
            x_validate=x_validate,
            y_validate=y_validate,
            display_interval=display_on_each_n_step
        )
        
        # After fitting, we can generate the predictions for visualization
        self._predict_full_range()

    def _predict_full_range(self):
        """Helper method to predict y values for the entire range of x."""
        X_full_for_prediction = [[val] for val in self.x]
        y_pred_values_nested = self.model.forward_batch(x=X_full_for_prediction)
        
        # Extract the scalar value from the output
        self.y_pred = np.array([val[0].value for val in y_pred_values_nested])


    def visualize_fit(self):
        """
        Visualizes the original function and the neural network's approximation.
        """
        if self.y_pred is None:
            print("Model has not been fitted yet. Please call the 'fit' method first.")
            return

        plt.figure(figsize=(12, 7))
        plt.plot(self.x, self.y, label='True Function', color='blue', linewidth=2)
        plt.plot(self.x, self.y_pred, label='NN Approximation', color='red', linestyle='--', alpha=0.7)
        plt.title('Neural Network Approximation of a Function')
        plt.xlabel('x')
        plt.ylabel('f(x)')
        plt.grid(True)
        plt.legend(fontsize=12)
        plt.axhline(0, color='black', linewidth=0.5)
        plt.axvline(0, color='black', linewidth=0.5)
        plt.show()

## Approximate Function 1

**Let's use Multilayer Perceptron to learn approximation for this function on interval $[0, 15]$**

$f(x) = -0.03 \cdot x^2 \cdot 0.05 \cdot \ln(x^3) + 0.5 \cdot \cos\left(\frac{x}{1.5}\right) + 0.1 \cdot \sin(5x)$


In [ ]:
def complicated_function(x):
    return -0.03 * x**2  * 0.05 * np.log(x ** 3) + 0.5 * np.cos(x/1.5) + 0.1 * np.sin(x*5)

x_values = np.linspace(0.1, 15, 100)
y_values = complicated_function(x_values)

model = Sequential(
    layers=[
        Dense(
            shape=(1, 32),  # More neurons in the first layer
            activation=Activation.TANH,
            initializer=Initializer(fill_type=InitializationType.GLOROT_NORMAL) # Try Glorot
        ),
        Dense(
            shape=(32, 16),
            activation=Activation.RELU,
            initializer=Initializer(fill_type=InitializationType.HE_NORMAL)
        ),
        Dense(
            shape=(16, 16),
            activation=Activation.RELU,
            initializer=Initializer(fill_type=InitializationType.HE_NORMAL)
        ),
        Dropout(0.4, 16), # Slightly lower dropout rate, placed after a block
        Dense(
            shape=(16, 8),
            activation=Activation.RELU,
            initializer=Initializer(fill_type=InitializationType.HE_NORMAL)
        ),
        Dense(
            shape=(8, 1),
            activation=Activation.LINEAR,
            initializer=Initializer(fill_type=InitializationType.HE_NORMAL)
        )
    ]
)


approximator1 = FuncApproximator(
    model=model,
    scalar_func=complicated_function,
    x=x_values,
    y=y_values
)
approximator1.fit(
    batch_size=16,
    test_split_size=0.1,
    epochs=50,
    display_on_each_n_step=5,
    learning_rate=0.002
)
approximator1.visualize_fit()

## Approximate Function 2


In [ ]:
def dampened_wave_function_2(x):
    """
    A function that represents a dampened wave.
    """
    return np.exp(-0.5 * x) * np.sin(x * 2)

x_values_2 = np.linspace(0, 10, 200)  # Using more points for a smoother curve
y_values_2 = dampened_wave_function_2(x_values_2)

model_2 = Sequential(
    layers=[
        Dense(
            shape=(1, 32),  # Increased neurons for more complexity
            activation=Activation.TANH,
            initializer=Initializer(fill_type=InitializationType.RANDOM_NORMAL)
        ),
        Dense(
            shape=(32, 16),
            activation=Activation.TANH,
            initializer=Initializer(fill_type=InitializationType.RANDOM_NORMAL)
        ),
        Dense(
            shape=(16, 16),
            activation=Activation.RELU,
            initializer=Initializer(fill_type=InitializationType.RANDOM_NORMAL)
        ),
        Dense(
            shape=(16, 1),
            activation=Activation.LINEAR,
            initializer=Initializer(fill_type=InitializationType.RANDOM_NORMAL)
        )
    ]
)

approximator2 = FuncApproximator(
    model=model_2,
    scalar_func=dampened_wave_function_2,
    x=x_values_2,
    y=y_values_2
)

approximator2.fit(
    batch_size=8,
    test_split_size=0.15,
    epochs=40,  # Increased epochs for better fitting
    display_on_each_n_step=1,
    learning_rate=0.005
)

approximator2.visualize_fit()